# Imports and utilities

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Transform service dependencies
from app.services.transform_service import TransformService
from app.models.transform import (
    TransformConstraints,
    TransformationType,
    TransformRequest,
    Recipe,
)

In [35]:
from shared.snowflake.client import SnowflakeClient

client = SnowflakeClient()

In [36]:
session = client.get_snowpark_session()

## Snowflake example usage

In [37]:
client.is_connected()

{'ok': True, 'version': '9.39.2'}

In [38]:
query = """
    SELECT 
        ID,
        NAME,
        INGREDIENTS,
        STEPS,
        MINUTES,
    FROM NUTRIRAG_PROJECT.RAW.RAW_RECIPES_10K
    LIMIT 1
"""

result_df = client.execute(query, fetch="one")

## Transform service example usage

In [46]:
transform_service = TransformService(session)

 Chargement des données PCA depuis CSV (ingredients_with_clusters.csv)...


[1.4-_load_pca_data] ✅ Données CSV chargées: 9318 ingrédients


### Input definition

In [47]:
ingredients = [
    "crabmeat",
    "cream cheese",
    "green onions",
    "garlic salt",
    "refrigerated crescent dinner rolls",
    "egg yolk",
    "water",
    "sesame seeds",
    "sweet and sour sauce",
]
steps = [
    "heat over to 375 degrees",
    "spray large cookie sheet with non-stick cooking spray",
    "in small bowl , combine crabmeat , cream cheese , onions and garlic salt and mix well",
    "unroll both cans of dough",
    "separate into 16 triangles",
    "cut each triangle in half lengthwise to make 32 triangles",
    "place 1 teaspoon crab mixture on center of each triangle about 1 inch from short side of triangle",
    "fold short ends of each triangle over filling",
    "pinch sides to seal",
    "roll up",
    "place on sprayed cookie sheet",
    "in small bowl , combine egg yolk and water and mix well",
    "brush egg mixture over snacks",
    "sprinkle with sesame seed",
    "bake at 375 degrees for 15 to 20 minutes or until golden brown",
    "serve warn snacks with sweet-and-sour sauce",
]

recipe = Recipe(
    id=94947,
    name="crab filled crescent snacks",
    serving_size=1,
    servings=4,
    ingredients=ingredients,
    quantity_ingredients=["1"] * len(ingredients),
    minutes=70.0,
    steps=steps,
    health_score=57
)
ingredients_to_remove = ["cream cheese"]

constraints = TransformConstraints(
    transformation=TransformationType.SUBSTITUTION,
    no_lactose=False,
    no_gluten=False,
    no_nuts=False,
    vegetarian=False,
    vegan=False,
    increase_protein=False,
    decrease_sugar=False,
    decrease_protein=False,
    decrease_carbs=False,
    decrease_calories=False,
    decrease_sodium=False,
)
# En principe, on récupère la request en entrée. La définition est juste pour l'exemple ici.
request = TransformRequest(
    recipe=recipe,
    ingredients_to_remove=ingredients_to_remove,
    constraints=constraints,
)
# Pour éviter de refaire les requêtes SQL sur les ingrédients de la recipe
cache = {
    "crabmeat": {
        "name": "crabapples raw",
        "matched_ingredient": "crabmeat",
        "protein": 0.4,
        "saturated_fats": 0.048,
        "fat": 0.3,
        "carbs": 19.95,
        "sodium": 1.0,
        "fiber": 0.0,
        "sugar": 0.0,
        "calories": 76.0,
    },
    "cream cheese": {
        "name": "cheese cream fat free",
        "matched_ingredient": "cream cheese",
        "protein": 15.69,
        "saturated_fats": 0.644,
        "fat": 1.0,
        "carbs": 7.66,
        "sodium": 702.0,
        "fiber": 0.0,
        "sugar": 5.48,
        "calories": 105.0,
    },
    "green onions": {
        "name": "onions frz whl ckd bld drnd wsalt",
        "matched_ingredient": "green onions",
        "protein": 0.71,
        "saturated_fats": 0.009000000000000001,
        "fat": 0.05,
        "carbs": 6.11,
        "sodium": 244.0,
        "fiber": 1.4,
        "sugar": 2.9,
        "calories": 26.0,
    },
    "garlic salt": {
        "name": "garlic  single clove  kashmir",
        "matched_ingredient": "garlic salt",
        "protein": 6.12,
        "saturated_fats": 0.03476,
        "fat": 0.16,
        "carbs": 23.46,
        "sodium": 8.87,
        "fiber": 4.01,
        "sugar": 3.73,
        "calories": 124.99700000000001,
    },
    "refrigerated crescent dinner rolls": {
        "name": "rolls dinner egg",
        "matched_ingredient": "refrigerated crescent dinner rolls",
        "protein": 9.5,
        "saturated_fats": 1.577,
        "fat": 6.4,
        "carbs": 52.0,
        "sodium": 566.0,
        "fiber": 3.7,
        "sugar": 4.3,
        "calories": 307.0,
    },
    "egg yolk": {
        "name": "egg yolk raw frz salted past",
        "matched_ingredient": "egg yolk",
        "protein": 14.07,
        "saturated_fats": 7.159,
        "fat": 22.93,
        "carbs": 1.77,
        "sodium": 3487.0,
        "fiber": 0.0,
        "sugar": 0.07,
        "calories": 275.0,
    },
    "water": {
        "name": "water melon  pale green",
        "matched_ingredient": "water",
        "protein": 0.59,
        "saturated_fats": 0.05033,
        "fat": 0.16,
        "carbs": 3.02,
        "sodium": 1.62,
        "fiber": 0.78,
        "sugar": 3.0,
        "calories": 16.73,
    },
    "sesame seeds": {
        "name": "sesame seeds whl rstdtstd",
        "matched_ingredient": "sesame seeds",
        "protein": 16.96,
        "saturated_fats": 6.722,
        "fat": 48.0,
        "carbs": 25.74,
        "sodium": 11.0,
        "fiber": 14.0,
        "sugar": 0.0,
        "calories": 565.0,
    },
    "sweet and sour sauce": {
        "name": "cherries sweet raw",
        "matched_ingredient": "sweet and sour sauce",
        "protein": 1.06,
        "saturated_fats": 0.038,
        "fat": 0.2,
        "carbs": 16.01,
        "sodium": 0.0,
        "fiber": 2.1,
        "sugar": 12.82,
        "calories": 63.0,
    },
    "turkey  white  rotisserie  deli cut": {
        "name": "turkey  white  rotisserie  deli cut",
        "matched_ingredient": "turkey  white  rotisserie  deli cut",
        "protein": 13.5,
        "saturated_fats": 0.11800000000000001,
        "fat": 3.0,
        "carbs": 7.7,
        "sodium": 1200.0,
        "fiber": 0.4,
        "sugar": 4.0,
        "calories": 112.0,
    },
}
transform_service.ingredients_cache = cache

### Transform service usage

In [ ]:
result = transform_service.transform(
    recipe=request.recipe,
    ingredients_to_remove=request.ingredients_to_remove,
    constraints=request.constraints,
)

Step 1 completed
🔍 Ingredient found: cheese cream fat free → cheese cream fat free
🎯 cheese cream fat free → soybeans mature seeds sprouted ckd stir-fried wsalt (PCA score: 0.253)
Step 2 completed
Step 3 completed
Step 6 completed
Step 7 completed


In [52]:
vars(result)

{'recipe': <app.models.transform.Recipe at 0x707bcff0b7d0>,
 'original_name': 'crab filled crescent snacks',
 'transformed_name': 'crab filled crescent snacks',
 'substitutions': None,
 'nutrition_before': <app.models.transform.NutritionDelta at 0x707be4b50c10>,
 'nutrition_after': <app.models.transform.NutritionDelta at 0x707be71429d0>,
 'success': True,
 'message': ''}

In [53]:
result.recipe.steps

['1. Heat over to 375 degrees.',
 '2. Spray large cookie sheet with non-stick cooking spray.',
 '3. In a small bowl, combine crabmeat, soybeans mature seeds sprouted ckd stir-fried wsalt, green onions, and garlic salt. Mix well.',
 '4. Unroll both cans of dough.',
 '5. Separate into 16 triangles.',
 '6. Cut each triangle in half lengthwise to make 32 triangles.',
 '7. Place 1 teaspoon crab mixture on the center of each triangle about 1 inch from the short side of the triangle.',
 '8. Fold short ends of each triangle over filling.',
 '9. Pinch sides to seal.',
 '10. Roll up.',
 '11. Place on sprayed cookie sheet.',
 '12. In a small bowl, combine egg yolk and water and mix well.',
 '13. Brush egg mixture over snacks.',
 '14. Sprinkle with sesame seeds.',
 '15. Bake at 375 degrees for 15 to 20 minutes or until golden brown. Note that the soybeans may provide a slightly different texture compared to the original cream cheese.',
 '16. Serve warm snacks with sweet-and-sour sauce.']

In [54]:
result.success

True

# Always close connexion before closing notebook

In [55]:
client.close()